<a href="https://colab.research.google.com/github/SattuSupCodes/SentimentAnalysis_TheoreticalLearning/blob/main/frustrating_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
!pip install --no-cache-dir \
  "torch==2.0.1+cpu" \
  "torchaudio==2.0.2+cpu" \
  --index-url https://download.pytorch.org/whl/cpu

!pip install "speechbrain==0.5.16" soundfile

Looking in indexes: https://download.pytorch.org/whl/cpu
ERROR: Could not find a version that satisfies the requirement torch==2.0.1+cpu (from versions: 2.2.0+cpu, 2.2.1+cpu, 2.2.2+cpu, 2.3.0+cpu, 2.3.1+cpu, 2.4.0+cpu, 2.4.1+cpu, 2.5.0+cpu, 2.5.1+cpu, 2.6.0+cpu, 2.7.0+cpu, 2.7.1+cpu, 2.8.0+cpu, 2.9.0+cpu, 2.9.1+cpu)
ERROR: No matching distribution found for torch==2.0.1+cpu


In [143]:
!pip install openai-whisper

In [144]:
import numpy as np
import librosa
import os
import torchaudio
import torch
import soundfile as sf
from sklearn.preprocessing import StandardScaler
from speechbrain.pretrained import EncoderClassifier
from speechbrain.lobes.models.ECAPA_TDNN import ECAPA_TDNN
from sklearn.cluster import AgglomerativeClustering
import whisper

In [145]:
device = "cuda" if torch.cuda.is_available() else "cpu"

ecapa = EncoderClassifier.from_hparams(
    source = "speechbrain/spkrec-ecapa-voxceleb",
    run_opts = {"device": device}
)
ecapa.eval()
print("ECAPA loaded (frozen embedding extractor)") #we downloaded our pretraind ecapa

ECAPA loaded (frozen embedding extractor)


In [146]:
wav, sr = sf.read("sample11.wav")
wav = torch.tensor(wav, dtype=torch.float32)
if wav.ndim == 2:
  wav = wav.mean(dim=1)
print(wav.shape, sr)

torch.Size([1048556]) 44100


In [164]:
model = whisper.load_model("base")
result = model.transcribe("sample11.wav")

segments = result["segments"]
for seg in segments:
    print(f"{seg['start']:.2f}-{seg['end']:.2f}: {seg['text']}")

/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


0.00-5.52:  Just that was kind of the thought and whatever and then you hit me and I was like no frickin way
5.52-9.40:  He's hitting me right now. So I was like absolutely. This is like perfect
9.40-15.50:  It's the universe just putting it in perfect timing because that makes me so happy because it was exactly the opposite way
15.50-21.76:  So I've for a long time had people that I'd love to sit down with and every time I saw you do something with mental health and
21.76-26.54:  Vogue or any of the segments that you ever did I was just like she's amazing like you know
26.54-29.66:  It's incredible to the way you were talking about it when you'd post about it
29.66-33.82:  I'd be like oh this is incredible like this is see you speaking so openly about it
33.82-39.74:  Invulnerably about it and so I've always wanted to do this and then but we've never really crossed paths
39.74-44.54:  And so I'm always just a bit like and I don't like asking friends of friends when I haven't met someo

# Since ONLY ECAPA is gving us he results, lets add MFCC layer

In [165]:
y, sr = librosa.load("sample11.wav", sr=None)

In [166]:
segment_len = int(1.5 * sr)
segments = [
    y[i:i+segment_len]
    for i in range(0, len(y), segment_len)
    if len(y[i:i+segment_len]) == segment_len
]


In [168]:
segment_len_sec = 1.5
segment_len = int(segment_len_sec * sr)

segments_audio = [
    y[i:i+segment_len]
    for i in range(0, len(y), segment_len)
    if len(y[i:i+segment_len]) == segment_len
]


In [169]:
def extract_mfcc(seg, sr):
    mfcc = librosa.feature.mfcc(y=seg, sr=sr, n_mfcc=13)
    return np.mean(mfcc, axis=1)

X = np.array([extract_mfcc(seg, sr) for seg in segments])


In [170]:
def mfcc_embed(seg, sr, n_mfcc=13):
    mfcc = librosa.feature.mfcc(y=seg, sr=sr, n_mfcc=n_mfcc)
    delta = librosa.feature.delta(mfcc)
    delta2 = librosa.feature.delta(mfcc, order=2)

    features = np.concatenate([
        np.mean(mfcc, axis=1),
        np.mean(delta, axis=1),
        np.mean(delta2, axis=1)
    ]) #lets just try making our mfcc stronger

    return features


In [172]:
def pitch_stats(seg, sr):
    pitch = librosa.yin(seg, fmin=50, fmax=400, sr=sr)
    pitch = pitch[~np.isnan(pitch)]
    if len(pitch) == 0:
        return [0, 0]
    return [np.mean(pitch), np.std(pitch)]

In [176]:
def full_embed(seg, sr):
    mfcc_feat = mfcc_embed(seg, sr)
    pitch_feat = pitch_stats(seg, sr)
    return np.concatenate([mfcc_feat, pitch_feat])


In [177]:
X = np.array([full_embed(seg, sr) for seg in segments_audio])
X = StandardScaler().fit_transform(X)

In [178]:
if segments_audio and len(segments_audio) > 0:
    # Use the mfcc_embed function defined earlier with a valid segment
    # For demonstration, we use the first segment from segments_audio
    sample_seg = segments_audio[0]
    mfcc_feat = mfcc_embed(sample_seg, sr)
    print("MFCC features for a sample segment:", mfcc_feat.shape)
else:
    print("No audio segments available to extract MFCC features.")



MFCC features for a sample segment: (39,)


#MFCC ENDS

In [179]:
if sr!=16000:
  wav = torchaudio.functional.resample(wav,sr,16000)
  sr=16000

In [180]:
print(sr)

16000


In [181]:
frame_sec = 0.025
frame_len = int(frame_sec*sr)
hop_len = frame_len//2

In [182]:
energies = []

for i in range(0, len(wav)-frame_len, hop_len):
  frame = wav[i:i+frame_len]
  energy = np.sum(frame.numpy()**2)
  energies.append(energy)
energies = np.array(energies)

In [183]:
energy_thresh = 0.03*np.max(energies)
speech_mask = energies>energy_thresh

In [184]:
segments = []
start = None

for i, is_speech in enumerate(speech_mask):
    t = i * hop_len / sr

    if is_speech and start is None:
        start = t

    elif not is_speech and start is not None:
        end = t
        if end - start > 0.5:
            segments.append((start, end))
        start = None


if start is not None:
    end = len(wav) / sr
    if end - start > 0.5:
        segments.append((start, end))

print("Segments created:", len(segments))



Segments created: 0


In [186]:
window_sec = 1.0 # Window length for ECAPA embedding
hop_sec = 0.02 # Hop length for sliding window over speech segments

win_len = int(window_sec * sr)
feat_hop_len = int(hop_sec * sr)

embeddings = []
features = [] # Retain the features list as it was present in the original cell

# --- Re-execute VAD segment creation with a modified threshold ---
# VAD parameters from upstream cells (kernel state):
vad_hop_len = 200  # From cell 2Z05uOFpasPB: frame_len // 2
vad_sr = 16000     # From cell _5vBQBqrakml: resampled sr

segments_for_extraction = [] # Use a new list to avoid conflict with the original 'segments' var
current_start = None
min_segment_duration_sec = 0.1 # Reduced from 0.5 seconds to capture shorter speech bursts

for i, is_speech in enumerate(speech_mask):
    t = i * vad_hop_len / vad_sr

    if is_speech and current_start is None:
        current_start = t

    elif not is_speech and current_start is not None:
        end = t
        if end - current_start > min_segment_duration_sec:
            segments_for_extraction.append((current_start, end))
        current_start = None

# Handle case where speech ends at the very end of the audio
if current_start is not None:
    end = len(wav) / vad_sr
    if end - current_start > min_segment_duration_sec:
        segments_for_extraction.append((current_start, end))

# Fallback if no segments are detected even with reduced threshold
if not segments_for_extraction:
    print("Warning: No speech segments detected even with reduced threshold. Using entire audio as a single segment.")
    segments_for_extraction = [(0, len(wav) / vad_sr)]


# --- Feature extraction loop ---
for start, end in segments_for_extraction: # Iterate over the newly created segments
    s = int(start * sr)
    e = int(end * sr)
    speech = wav[s:e]

    if len(speech) < win_len:
        continue # Skip segments shorter than the embedding window

    for i in range(0, len(speech) - win_len + 1, feat_hop_len):
        chunk = speech[i:i+win_len]

        with torch.no_grad():
            emb = ecapa.encode_batch(chunk.unsqueeze(0))

        emb = emb.squeeze(0)
        embeddings.append(emb.cpu().numpy())

# Only stack if embeddings list is not empty to prevent ValueError
if embeddings:
    embeddings = np.vstack(embeddings)
    print(embeddings.shape)
else:
    print("No embeddings could be generated from any segments after processing.")


No embeddings could be generated from any segments after processing.


In [187]:
print("Number of VAD segments:", len(segments))
print("First 10 VAD segments (sec):", segments[:10])

durations = [(end - start) for start, end in segments]
print("First 10 segment durations:", durations[:10])
print("Max segment duration:", max(durations) if durations else 0)


Number of VAD segments: 0
First 10 VAD segments (sec): []
First 10 segment durations: []
Max segment duration: 0


In [188]:
print(type(emb), emb.shape)

<class 'torch.Tensor'> torch.Size([1, 192])


In [189]:
clusterer = AgglomerativeClustering(
    n_clusters=None,
    metric="cosine",
    linkage="average",
    distance_threshold=0.3
)

labels = clusterer.fit_predict(embeddings)
print("Speakers detected:", len(set(labels)))

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
segment_feature = np.concatenate([emb.cpu().numpy().flatten(), mfcc_feat])
# shape: (231,)

In [ ]:
features.append(segment_feature)


In [ ]:
X = np.vstack(features)
print(X.shape)


In [ ]:
X = StandardScaler().fit_transform(X)

In [ ]:
clusterer = AgglomerativeClustering(
    n_clusters=None,
    distance_threshold=0.65,
    metric="cosine",
    linkage="average"
)

labels = clusterer.fit_predict(X)
print("Speakers detected:", len(set(labels)))
